# Titanic: Machine Learning from Disaster

In [1]:
import numpy as np  
import pandas as pd  
from sklearn import tree

# 基本の描画ライブラリ（２つ）
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# 便利な設定

# pandasで全ての列を表示
pd.options.display.max_columns = None

# 図をipython notebook内で表示
%matplotlib inline

# DeplicatedWarningを避けるため
import warnings
#warnings.filterwarnings("ignore", category=DeprecationWarning)

## データの読み込み

In [3]:
df_train = pd.read_csv("data/train.csv")
df_test = pd.read_csv("data/test.csv")

## データ整形

目的：データの特徴を理解し、機械学習ができる形に整形する

ここで俺氏、条件に応じて新規カラムに数値データを作成する方法を忘れる  
→Meetup15の資料見返し

え、mapの引数でDict渡すとそれだけで文字データから数値データに変換してくれるの？

# .map() SUGEEEEEEEEEEEEEEEEEEEEEEEE

## Sexの変換

In [1]:
Sex_typeId = {'male': 0, 'female': 1}

In [2]:
df_train["SexId"] = df_train['Sex'].map(Sex_typeId)
### .mapを用いてSexを0と1のデータに変換

NameError: name 'df_train' is not defined

In [6]:
df_train["SexId"].head(15)

0     0
1     1
2     1
3     1
4     0
5     0
6     0
7     0
8     1
9     1
10    1
11    1
12    0
13    0
14    1
Name: SexId, dtype: int64

## Embarkedの変換

In [7]:
Embarked_typeId = {'S': 0, 'C': 1, 'Q': 2}

In [8]:
df_train["Embarked"] = df_train["Embarked"].fillna("S")
# NaN値のところはfillnaメソッドを使ってSで埋める

In [9]:
df_train["EmbarkedId"] = df_train['Embarked'].map(Embarked_typeId)

In [10]:
df_train["EmbarkedId"].head(15)

0     0
1     1
2     0
3     0
4     0
5     2
6     0
7     0
8     0
9     1
10    0
11    0
12    0
13    0
14    0
Name: EmbarkedId, dtype: int64

## AgeのNaN値を全体の平均値で埋める

In [11]:
df_train["Age"].mean()

29.69911764705882

In [12]:
df_train["Age"] = df_train["Age"].fillna(df_train["Age"].mean())

## FareのNaN値を全体の平均値で埋める

In [13]:
df_train["Fare"] = df_train["Fare"].fillna(df_train["Fare"].mean())

### 説明変数と目的変数を作成

In [14]:
target = df_train["Survived"].values
# 目的変数

In [15]:
df_train.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'SexId', 'EmbarkedId'],
      dtype='object')

In [16]:
features_one = df_train[["SexId", "Age", "Pclass", "Fare", "EmbarkedId"]].values
# 説明変数（使う変数をデータフレームの中からカラム名で指定する）

In [17]:
my_tree_one = tree.DecisionTreeClassifier()
# DecisionTreeClassifierを初期化

In [18]:
my_tree_one = my_tree_one.fit(features_one, target)
# 説明変数と目的変数をnumpy.arrayで投げる
# 質問：clf.fit(X,Y)これってやっぱり説明変数がＸで目的変数がＹ？逆じゃだめ？

In [19]:
my_tree_one.feature_importances_

array([ 0.31117584,  0.24748133,  0.12444517,  0.29392899,  0.02296867])

In [20]:
my_tree_one.score(features_one, target)

0.97979797979797978

### 正答率81%（´・ω・｀）でも前の分類器（76%）よりはいいのでTestデータ使ってPredictしてみる

## まずは焦らずtestデータの変換から

In [21]:
df_test["SexId"] = df_test['Sex'].map(Sex_typeId)
### .mapを用いてtestデータのSexを0と1のデータに変換

In [22]:
df_test["SexId"].head(15)

0     0
1     1
2     0
3     0
4     1
5     0
6     1
7     0
8     1
9     0
10    0
11    0
12    1
13    0
14    1
Name: SexId, dtype: int64

In [23]:
df_test["Embarked"] = df_test["Embarked"].fillna("S")
# NaN値のところはfillnaメソッドを使ってSで埋める

In [24]:
df_test["EmbarkedId"] = df_test['Embarked'].map(Embarked_typeId)

In [25]:
df_test["Age"] = df_test["Age"].fillna(df_test["Age"].mean())

In [26]:
df_test[["SexId", "Age", "Pclass", "Fare", "EmbarkedId"]].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 0 to 417
Data columns (total 5 columns):
SexId         418 non-null int64
Age           418 non-null float64
Pclass        418 non-null int64
Fare          417 non-null float64
EmbarkedId    418 non-null int64
dtypes: float64(2), int64(3)
memory usage: 19.6 KB


In [27]:
df_test["Fare"] = df_test["Fare"].fillna(df_test["Fare"].mean())

---

データの変換終わり

---

In [28]:
test_features = df_test[["SexId", "Age", "Pclass", "Fare", "EmbarkedId"]].values
test_features

array([[  0.        ,  34.5       ,   3.        ,   7.8292    ,   2.        ],
       [  1.        ,  47.        ,   3.        ,   7.        ,   0.        ],
       [  0.        ,  62.        ,   2.        ,   9.6875    ,   2.        ],
       ..., 
       [  0.        ,  38.5       ,   3.        ,   7.25      ,   0.        ],
       [  0.        ,  30.27259036,   3.        ,   8.05      ,   0.        ],
       [  0.        ,  30.27259036,   3.        ,  22.3583    ,   1.        ]])

In [29]:
# Make your prediction using the test set
my_prediction = my_tree_one.predict(test_features)
my_prediction

array([0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0,
       0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0,
       1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0,

# 【解決済み】なんでSurvivedが全部１なん・・・・・

- ２つ↑くらいのカラムで作ったmy_predictionのデータ設計(使用するデータの選択)が間違ってました。。。

In [30]:
# Create a data frame with two columns: PassengerId & Survived.
# Survived contains your predictions
PassengerId =np.array(df_test["PassengerId"]).astype(int)
my_solution = pd.DataFrame(my_prediction, PassengerId, columns = ["Survived"])

In [31]:
# Check that your data frame has 418 entries
print(my_solution.shape)

(418, 1)


In [32]:
my_solution.head()
# データの形があってるか確認

,Survived
892,0
893,0
894,1
895,1
896,1


### テストデータを使ってPredictしたデータで提出用のcsvファイルを作成

In [33]:
# Output submmision data
my_solution.to_csv("output/submission03.csv", index_label = ["PassengerId"])

![title](img/Submission03_rank.png)

# 2966位！！！！！

- Submission01の4439位から1473位ランクアップ

くそぅ。中身についてはあんまりわかってないから一旦Decision treeの勉強だ！  
http://pythondatascience.plavox.info/scikit-learn/scikit-learn%E3%81%A7%E6%B1%BA%E5%AE%9A%E6%9C%A8%E5%88%86%E6%9E%90/

わかったこと  
- Decision tree Ananlysysにはいくつかの手法が存在
- scikit-learnではCART法というのを使っている

## 今後の課題

- モデルデータがオーバーフィッティングしてるとかしてないとかどうやって調べるんだろう？
- 他のカラムのデータもまとめて突っ込んだら精度あがる？
- 他のアルゴリズムの方が精度上がる？

# 今回は以上(๑•̀ㅂ•́)و✧